In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm

dfs = []

dfs.append(pd.read_csv('./submission/2023-01-04-1749.csv'))
dfs.append(pd.read_csv('./submission/2023-01-04-0211.csv'))
dfs.append(pd.read_csv('./submission/2023-01-05-1454_crf.csv'))
dfs.append(pd.read_csv('./submission/2023-01-05-0947.csv'))

if len(dfs) < 3:
    print("At least 3 csv files are requied!")

In [18]:
submission = pd.DataFrame()
submission['image_id'] = dfs[0]['image_id']

In [19]:
PredictionString = []

for idx in tqdm(range(len(dfs[0]))):
    image_segmentation_preds = []
    for df in dfs:
        image_segmentation_preds.append(df['PredictionString'][idx].split(' '))

    result = ''

    for i in range(len(aa)):
        pixel_dict = {'0' : 0, '1' : 0, '2' : 0, 
                      '3' : 0, '4' : 0, '5' : 0,
                      '6' : 0, '7' : 0, '8' : 0,
                      '9' : 0, '10' : 0}

        best_model = image_segmentation_preds[0][i]
        second_model = image_segmentation_preds[1][i]
        third_model = image_segmentation_preds[2][i]
        
        for image_segmentation_pred in image_segmentation_preds:
            pixel_dict[image_segmentation_pred[i]] += 1

        # 최댓값을 갖는 키값 추출
        pixel = [key for key, value in pixel_dict.items() if value == max(pixel_dict.values())]
        
        # ensemble 모델이 ~4개
        # 만약 최댓값을 갖는 키가 2개 이상(같은 표를 가진 경우)이고,
        # best_model이 영향력을 행사했다면
        if len(pixel) >= 2 and best_model in pixel:
            result += best_model + ' '
        
        # ensemble 모델이 5개~6개
        # 만약 최댓값을 갖는 키가 2개 이상(같은 표를 가진 경우)이고,
        # best_model이 outsider, second_model이 영향력을 행사했다면
        elif len(pixel) >= 2 and second_model in pixel:
            result += second_model + ' '
            
        # 만약 1개라면 해당 키 반환
        else:
            result += pixel[0] + ' '

    # 마지막 공백 제거
    result = result[:-1]
    PredictionString.append(result)

100%|██████████| 819/819 [06:59<00:00,  1.95it/s]


In [20]:
submission['PredictionString'] = PredictionString

In [21]:
submission.to_csv('./hard_voting_models.csv', index=False)